In [1]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    model_path: Path 
    dataset: Path 
    epochs: int 
    imgsz: tuple
    batch_size: int 
    lr: float 
    weights: str 
    channels: int

In [5]:
from LungColonClassifier.constants import *
from LungColonClassifier.utils.common import read_yaml, create_directories

2024-02-02 07:40:36.562204: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 07:40:40.518129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 07:40:40.518434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 07:40:41.024105: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-02 07:40:42.331881: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-02 07:40:51.296315: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        config = self.config.training 
        params = self.params 
        create_directories([config.root_dir])
        
        training_config = TrainingConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            dataset="artifacts/data_ingestion/lung_colon_image_set/",
            epochs=params.EPOCHS,
            imgsz=params.IMG_SZ,
            weights=params.WEIGHTS,
            channels=params.CHANNELS,
            lr=params.LEARNING_RATE,
            batch_size=params.BATCH_SIZE
        )
        return training_config

In [9]:
import pandas as pd 
import os
import tensorflow as tf
from tensorflow.keras.applications import Xception 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau 
from tensorflow.keras.models import Model 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-02-01 18:20:30.391385: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 18:20:31.168224: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 18:20:31.168302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 18:20:31.512347: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 18:20:32.389656: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 18:20:40.855365: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 


  

    def get_callbacks(self):
        callbacks = []

        # ModelCheckpoint
        checkpoint = ModelCheckpoint(filepath=self.config.model_path, verbose=1, monitor='val_accuracy', mode='max') 
        callbacks.append(checkpoint) 

        # Import ReduceLROnPlateau if not imported earlier
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
        callbacks.append(reduce_lr)

        # Import EarlyStopping if not imported earlier
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
        callbacks.append(early_stopping)

        return callbacks 
        
    def train(self):
        data = self.load_data()
        train_gen, valid_gen, test_gen = self.process_data(data)
        

        img_size = (224,224)
        channels = 3
        img_shape = (img_size[0], img_size[1], channels)
        class_count = len(list(train_gen.class_indices.keys()))


        # Create Xception base model
        base_model = Xception(input_shape=img_shape, include_top=False, weights='imagenet')

        base_model.trainable = True

        x = base_model.output

        x = GlobalAveragePooling2D()(x)

        y = Dense(256, activation='relu')(x)

        predictions = Dense(class_count, activation='softmax', name='final')(y)

        model_Xception = Model(inputs=base_model.input, outputs=predictions)  


        model_Xception.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy']) 


        callbacks=self.get_callbacks() 
        model_Xception.fit(train_gen,epochs=10,validation_data=valid_gen,callbacks=[callbacks])
        
        

In [12]:
try:
    config = ConfigurationManager()
    train_config = config.get_training_config()
    train = Training(config=train_config)
    train.train()
except Exception as e:
    raise e

[2024-02-01 18:21:41,463: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-01 18:21:41,570: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-01 18:21:41,573: INFO: common: created directory at: artifacts]
[2024-02-01 18:21:41,574: INFO: common: created directory at: artifacts/training]
Found 20000 validated image filenames belonging to 5 classes.
Found 2500 validated image filenames belonging to 5 classes.
Found 2500 validated image filenames belonging to 5 classes.
Epoch 1/10


KeyboardInterrupt: 